<a href="https://colab.research.google.com/github/asishk01/GIS/blob/main/interactive_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploades = files.upload()

Saving GWQ.csv to GWQ.csv
Saving VSKP_VILLAGES.cpg to VSKP_VILLAGES.cpg
Saving VSKP_VILLAGES.dbf to VSKP_VILLAGES.dbf
Saving VSKP_VILLAGES.prj to VSKP_VILLAGES.prj
Saving VSKP_VILLAGES.sbn to VSKP_VILLAGES.sbn
Saving VSKP_VILLAGES.sbx to VSKP_VILLAGES.sbx
Saving VSKP_VILLAGES.shp to VSKP_VILLAGES.shp
Saving VSKP_VILLAGES.shp.xml to VSKP_VILLAGES.shp.xml
Saving VSKP_VILLAGES.shx to VSKP_VILLAGES.shx


In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np
from pykrige.ok import OrdinaryKriging
import rasterio
from rasterio.features import geometry_mask
from affine import Affine
import folium
from folium import plugins
from branca.colormap import LinearColormap
import rasterio.mask

boundary = gpd.read_file("VSKP_VILLAGES.shp")

df = pd.read_csv("GWQ.csv")
geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
points = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

points = points.to_crs(boundary.crs)

x = points.geometry.x.values
y = points.geometry.y.values
values = points['PH '].values
valid = ~np.isnan(values)
x, y, values = x[valid], y[valid], values[valid]

minx, miny, maxx, maxy = boundary.total_bounds
grid_res = 300
gridx = np.linspace(minx, maxx, grid_res)
gridy = np.linspace(miny, maxy, grid_res)

OK = OrdinaryKriging(x, y, values, variogram_model='spherical', verbose=False)
z, ss = OK.execute('grid', gridx, gridy)

transform = Affine((maxx - minx) / (grid_res - 1), 0, minx, 0, (maxy - miny) / (grid_res - 1), miny)
mask = geometry_mask([geom for geom in boundary.geometry], transform=transform, invert=True, out_shape=z.shape)
z_masked = np.where(mask, z, np.nan)

out_tif = "kriged_ph.tif"
with rasterio.open(out_tif, 'w', driver='GTiff',
                   height=z_masked.shape[0], width=z_masked.shape[1],
                   count=1, dtype=z_masked.dtype,
                   crs=boundary.crs, transform=transform,
                   nodata=np.nan) as dst:
    dst.write(z_masked, 1)

def extract_mean_value(geom, raster_path):
    with rasterio.open(raster_path) as src:
        try:
            out_image, _ = rasterio.mask.mask(src, [geom], crop=True, nodata=np.nan)
            return np.nanmean(out_image[0])
        except Exception:
            return np.nan

boundary["kriged_PH"] = boundary.geometry.apply(lambda g: extract_mean_value(g, out_tif))

geojson = boundary.to_crs(epsg=4326)

min_ph = geojson['kriged_PH'].min()
max_ph = geojson['kriged_PH'].max()

colormap = LinearColormap(
    colors=['#00e400', '#ffff00', '#ff7e00', '#ff0000', '#800000'],
    vmin=min_ph,
    vmax=max_ph,
    caption='Groundwater pH Levels'
)

def style_function(feature):
    ph = feature['properties']['kriged_PH']
    return {
        'fillColor': colormap(ph) if ph is not None and not np.isnan(ph) else 'grey',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    }

centroid = geojson.to_crs(epsg=3857).geometry.centroid.to_crs(epsg=4326).iloc[0]
center = [centroid.y, centroid.x]

m = folium.Map(location=center, zoom_start=10, tiles=None, control_scale=True)

folium.TileLayer(
    tiles='http://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
    attr='Google',
    name='Satellite',
    overlay=False,
    control=True
).add_to(m)

folium.TileLayer(
    tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
    attr='© OpenStreetMap contributors',
    name='Default',
    overlay=False,
    control=True
).add_to(m)

fg = folium.FeatureGroup(name="pH")
folium.GeoJson(
    geojson,
    style_function=style_function,
    popup=folium.GeoJsonPopup(fields=["kriged_PH"], aliases=["pH:"])
).add_to(fg)
fg.add_to(m)

from folium import MacroElement
from jinja2 import Template

class Legend(MacroElement):
    _template = Template("""
        {% macro html(this, kwargs) %}
        <div id="legend" style="
            position: absolute;
            bottom: 20px;
            left: 50%;
            transform: translateX(-50%);
            z-index: 9999;
            background: white;
            border-radius: 8px;
            padding: 10px 14px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.3);
            font-family: Arial, sans-serif;
            width: 300px;
            font-size: 14px;
        ">
            <div style="font-weight: bold;">Groundwater pH Levels</div>
            <div style="font-size: 12px; color: #555; margin-top: 2px;">
                Tap on map to see more info
            </div>
            <div style="margin-top: 12px;">
                <div style="
                    height: 14px;
                    background: linear-gradient(to right, #00e400, #ffff00, #ff7e00, #ff0000, #800000);
                    border: 1px solid #ccc;
                    border-radius: 4px;
                "></div>
                <div style="
                    display: flex;
                    justify-content: space-between;
                    font-size: 11px;
                    margin-top: 4px;
                    color: #333;
                ">
                    <span>7.8</span><span>8.5</span><span>8.65</span><span>9.1</span><span>9.5</span>
                </div>
            </div>
        </div>
        {% endmacro %}
    """)

    def __init__(self):
        super().__init__()
        self._name = 'Legend'

m.add_child(Legend())

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = plugins.MousePosition(
    position="bottomright",
    separator=" | ",
    empty_string="Unavailable",
    lng_first=True,
    num_digits=5,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)
mouse_position.add_to(m)

plugins.Fullscreen(position='topright').add_to(m)
minimap = plugins.MiniMap(toggle_display=True)
m.add_child(minimap)

folium.LayerControl(position='topright', collapsed=True).add_to(m)

m.save("kriged_ph_interactive_map.html")
print("Map saved as kriged_ph_interactive_map.html")
m


Map saved as kriged_ph_interactive_map.html


In [ ]:
from google.colab import files
files.download("Interactive_map.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>